In [33]:
from dotenv import load_dotenv

load_dotenv()

True

## Propm template 🧐

In [32]:
from langchain.prompts import PromptTemplate

propmt = PromptTemplate.from_template("Describe an object that you find {adjective} and why it has effect on you.")

In [34]:
propmt.format(adjective="awful")

'Describe an object that you find awful and why it has effect on you.'

In [35]:
template = "you are useful assistent that translates from {language_input} to {language_output} of text: {text}"
template

'you are useful assistent that translates from {language_input} to {language_output} of text: {text}'

In [43]:
text = "Me encanta programar y la plata"

In [37]:
prompt_template = PromptTemplate(
    input_variables=["language_input", "language_output", "text"],
    template=template,
)

## Wrapper Wikipedia 🌍

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [24]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [15]:
print(tool.name)
print(tool.description)
print(tool.args)

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
{'query': {'title': 'Query', 'description': 'query to look up on wikipedia', 'type': 'string'}}


In [25]:
tool.run({ 'query': "what is the purpose of cryptocurrencies" })

'Page: Cryptocurrency\nSummary: A cryptocurrency, crypto-currency, or crypto is a digital currency designed to work as a medium of exchange through a computer network that is not reliant on any central authority, such as a government or bank, to uphold or maintain it. \nIndividual coin ownership records are stored in a digital ledger, which is a computerized database using strong cryptography to secure transaction records, control the creation of additional coins, and verify the transfer of coin ow'

## Tavily Search 🔍

In [31]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
search.invoke({ 'query': "what is the temperature in piedecuesta Santander Colombia" })

[{'url': 'https://www.accuweather.com/en/co/piedecuesta/106558/weather-forecast/106558',
  'content': 'Dust & Dander Very High. World South America Colombia Santander Piedecuesta. Piedecuesta, Santander, Colombia Weather Forecast, with current conditions, wind, air quality, and what to expect for ...'},
 {'url': 'https://www.wunderground.com/forecast/co/piedecuesta',
  'content': 'Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Piedecuesta area. ... Piedecuesta, Santander, Colombia 10-Day Weather ...'},
 {'url': 'https://www.wunderground.com/weather/co/piedecuesta',
  'content': 'Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Piedecuesta area. ... Piedecuesta, Santander, Colombia Weather ...'},
 {'url': 'https://weather.com/weather/tenday/l/Piedecuesta+Santander+Colombia?canonicalCityId=cd837aca110e42132aa00708f506bdb417893d32eae4f

## Calling LLM 📞

In [38]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

In [45]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

chain = LLMChain(llm=llm, prompt=prompt_template)

In [46]:
answer = chain.invoke(
    input={
        "language_input": "spanish",
        "language_output": "english",
        "text": text
    }
)

print(answer)

{'language_input': 'spanish', 'language_output': 'english', 'text': 'I love programming and money'}
